In [2]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

from collections import deque

def make_binary_tree(nums):
    if not nums:
        return TreeNode(None)
    
    tree = TreeNode(nums[0])
    q = deque([tree])
    i = 1
    while i < len(nums):
        node = q.popleft()
        node.left = TreeNode(val=nums[i])
        q.append(node.left)
        if i + 1 < len(nums):
            node.right = TreeNode(val=nums[i+1])
            q.append(node.right)
        i += 2

    return tree

def print_binary_tree(root):
    if root:
        print(root.val, end=" ")
        print_binary_tree(root.left)
        print_binary_tree(root.right)

print_binary_tree(make_binary_tree([2,1,4]))
print()
print_binary_tree(make_binary_tree([1,None,8]))
print()
print_binary_tree(make_binary_tree([3,9,20,None,None,15,7]))

2 1 4 
1 None 8 
3 9 None None 20 15 7 

In [4]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def make_linked_list(nums):
    linked_list = ListNode(val=nums[0])
    node = linked_list
    for n in nums[1:]:
        node.next = ListNode(val=n)
        node = node.next
    return linked_list

def print_linked_list(node):
    while node:
        print(node.val, end=" ")
        node = node.next
    print()

print_linked_list(make_linked_list([2,4,3]))
print_linked_list(make_linked_list([5,6,4]))

2 4 3 
5 6 4 


**95. Unique Binary Search Trees II**

Given an integer n, return all the structurally unique BST's (binary search trees), which has exactly n nodes of unique values from 1 to n. Return the answer in any order.

https://leetcode.com/problems/unique-binary-search-trees-ii/

In [98]:
# slow recursive, via https://www.youtube.com/watch?v=A7OII4X4cw0&ab_channel=HappyCoding
# easy to make faster using cache
from functools import cache
def generateTrees(n: int) -> list[TreeNode]:
    @cache
    def dfs(nums): 
        if not nums:
            return [None]
        
        ans = []

        for i in range(len(nums)):
            left_trees = dfs(nums[:i])
            right_trees = dfs(nums[i+1:])

            for l in left_trees:
                for r in right_trees:
                    root = TreeNode(nums[i])
                    root.left = l
                    root.right = r
                    ans.append(root)

        return ans
    nums = tuple(range(1, n+1))

    return dfs(nums)

# iterative, via https://leetcode.com/problems/unique-binary-search-trees-ii/discuss/31586/Iterative-Python-solution-DP-76-ms
def generateTrees(n: int) -> list[TreeNode]:
    if not n:
        return []
    
    dp = [[[None] for _ in range(n+1)] for _ in range(n+2)]

    for l in range(1, n+1):
        for i in range(1, n-l+2):
            j = i + l - 1
            dp[i][j] = []
            for k in range(i, j+1):
                left = dp[i][k-1]
                right = dp[k+1][j]
                for lt in left:
                    for rt in right:
                        root = TreeNode(k, left=lt, right=rt)
                        dp[i][j].append(root)
    
    return dp[1][n]


**110. Balanced Binary Tree**

Given a binary tree, determine if it is height-balanced.

https://leetcode.com/problems/balanced-binary-tree/

In [7]:
# via neetcode https://www.youtube.com/watch?v=QfJsau0ItOY&t=282s&ab_channel=NeetCode
def isBalanced(root: TreeNode) -> bool:
    def dfs(node):
        if not node:
            return True, 0

        right = dfs(node.right)
        left = dfs(node.left)

        return right[0] and left[0] and abs(right[1] - left[1]) <= 1, max(right[1], left[1]) + 1

    return dfs(root)[0]

print(isBalanced(make_binary_tree([3,9,20,None,None,15,7])))


True


**113. Path Sum II**

Given the root of a binary tree and an integer targetSum, return all root-to-leaf paths where the sum of the node values in the path equals targetSum. Each path should be returned as a list of the node values, not node references.

A root-to-leaf path is a path starting from the root and ending at any leaf node. A leaf is a node with no children.

https://leetcode.com/problems/path-sum-ii/

In [7]:
def pathSum(root: TreeNode, targetSum: int) -> list[list[int]]:
    if not root:
        return []
    ans = []
    def helper(node, total, arr):
        if not node.left and not node.right:
            if total + node.val == targetSum:
                ans.append(arr + [node.val])
    
        else:
            if node.left:
                helper(node.left, total + node.val, arr + [node.val])
            if node.right:
                helper(node.right, total + node.val, arr + [node.val])

    helper(root, 0, [])

    return ans

**114. Flatten Binary Tree to Linked List**

Given the root of a binary tree, flatten the tree into a "linked list":

* The "linked list" should use the same TreeNode class where the right child pointer points to the next node in the list and the left child pointer is always null.
* The "linked list" should be in the same order as a pre-order traversal of the binary tree.

https://leetcode.com/problems/flatten-binary-tree-to-linked-list/

In [2]:
# my solution, a little slow as it's done in two passes with an auxillary queue.
def flatten(root: TreeNode) -> None:
    """
    Do not return anything, modify root in-place instead.
    """
    if not root:
        return root
    q = deque([])
    def preorder(node):
        if not node:
            return
        
        q.append(node)
        preorder(node.left)
        preorder(node.right)
    
    preorder(root)
    
    node = q.popleft()
    node.left = None
    while q:
        node.right = q.popleft()
        node = node.right
        node.left = None
        
    return root

**128. Longest Consecutive Sequence**

Given an unsorted array of integers nums, return the length of the longest consecutive elements sequence.

You must write an algorithm that runs in O(n) time.

https://leetcode.com/problems/longest-consecutive-sequence/

In [58]:
# union find - O(n log n)
def longestConsecutive(nums: list[int]) -> int:
    if not nums: 
        return 0

    max_l = 1
    parents = {n:[n,1] for n in nums}
    
    def find(n):
        nn = n
        while nn != parents[nn][0]:
            nn = parents[nn][0]
        parents[n] = parents[nn]
        return parents[n]
        
    def union(m, n):
        pm, lm = find(m)
        pn, ln = find(n)
        parents[pm][0] = pn
        parents[n][1] = lm+ln
        nonlocal max_l
        max_l = max(max_l, parents[n][1])
        
    for n in parents.keys():
        if n+1 in parents:
            union(n, n+1)
            
    return max_l

# sorting - O(n log n)
def longestConsecutive(nums: list[int]) -> int:
    if not nums: 
        return 0
    
    nums.sort()
    current = 1
    longest = 1
    for i in range(1, len(nums)):
        if nums[i] - 1 == nums[i-1]:
            current += 1
            longest = max(longest, current)
        elif nums[i] != nums[i-1]:
            current = 1

    return longest

# O(n)
def longestConsecutive(nums: list[int]) -> int:
    nums = set(nums)
    longest = 0

    for n in nums:
        if n-1 not in nums:
            length = 0
            while n + length in nums:
                length += 1
            longest = max(longest, length)

    return longest


print(longestConsecutive([100,4,200,1,3,2])) # 4
print(longestConsecutive([0,3,7,2,5,8,4,6,0,1])) # 9
print(longestConsecutive([1,2,0,1])) # 3
print(longestConsecutive([4,0,-4,-2,2,5,2,0,-8,-8,-8,-8,-1,7,4,5,5,-4,6,6,-3])) # 5

4
9
3
5


**318. Maximum Product of Word Lengths**

Given a string array words, return the maximum value of length(word[i]) * length(word[j]) where the two words do not share common letters. If no such two words exist, return 0.

 https://leetcode.com/problems/maximum-product-of-word-lengths/

In [8]:
def maxProduct(words: list[str]) -> int:
    d = {}
    for w in words:
        t = tuple(sorted(set(w)))
        d[t] = max(d.get(t,0), len(w))
    maximum = 0
    arr = [[set(k), v] for k, v in d.items()]
    for i in range(len(arr)-1):
        for j in range(i+1, len(arr)):
            if not (arr[i][0] & arr[j][0]):
                maximum = max(maximum, arr[i][1]*arr[j][1])
    return maximum

print(maxProduct(["abcw","baz","foo","bar","xtfn","abcdef"])) # 16
print(maxProduct(["a","ab","abc","d","cd","bcd","abcd"])) # 4
print(maxProduct(["a","aa","aaa","aaaa"])) # 0

16
4
0


**399. Evaluate Division**

You are given an array of variable pairs equations and an array of real numbers values, where equations[i] = [Ai, Bi] and values[i] represent the equation Ai / Bi = values[i]. Each Ai or Bi is a string that represents a single variable.

You are also given some queries, where queries[j] = [Cj, Dj] represents the jth query where you must find the answer for Cj / Dj = ?.

Return the answers to all queries. If a single answer cannot be determined, return -1.0.

Note: The input is always valid. You may assume that evaluating the queries will not result in division by zero and that there is no contradiction.

https://leetcode.com/problems/evaluate-division/

In [31]:
# dfs
def calcEquation(equations: list[list[str]], values: list[float], queries: list[list[str]]) -> list[float]:
    unique = set([v for e in equations for v in e])
    d = {}
    for v in unique:
        d[v] = {v:1}

    for i in range(len(equations)):
        d[equations[i][0]][equations[i][1]] = values[i]
        d[equations[i][1]][equations[i][0]] = 1/values[i]


    ans = []

    def dfs(start, end):
        nonlocal d
        visited = {start}
        stack = [(start, 1)]
        while stack:
            v, m = stack.pop()
            if v not in d:
                return -1
            for k, mm in d[v].items():
                if k == end:
                    return m * mm
                else:
                    if k not in visited:
                        stack.append((k, m*mm))
                    visited.add(k)
        return -1
    
    for q in queries:
        ans.append(dfs(q[0], q[1]))

    return ans

# Floyd–Warshall - via https://leetcode.com/problems/evaluate-division/discuss/3544428/Python-Elegant-and-Short-or-FloydWarshall
from collections import defaultdict
def calcEquation(equations: list[list[str]], values: list[float], queries: list[list[str]]) -> list[float]:
    graph = defaultdict(dict)

    for (u, v), val in zip(equations, values):
        graph[u][u] = graph[v][v] = 1
        graph[u][v] = val
        graph[v][u] = 1 / val

    for k in graph:
        for i in graph[k]:
            for j in graph[k]:
                graph[i][j] = graph[i][k] * graph[k][j]

    return [graph[u].get(v, -1) for u, v in queries]



print(calcEquation(equations = [["a","b"],["b","c"]], values = [2.0,3.0], queries = [["a","c"],["b","a"],["a","e"],["a","a"],["x","x"]])) # [6.00000,0.50000,-1.00000,1.00000,-1.00000]
print(calcEquation(equations = [["a","b"],["b","c"],["bc","cd"]], values = [1.5,2.5,5.0], queries = [["a","c"],["c","b"],["bc","cd"],["cd","bc"]])) # [3.75000,0.40000,5.00000,0.20000]
print(calcEquation(equations = [["a","b"]], values = [0.5], queries = [["a","b"],["b","a"],["a","c"],["x","y"]])) # [0.50000,2.00000,-1.00000,-1.00000]

[6.0, 0.5, -1, 1.0, -1]
[3.75, 0.4, 5.0, 0.2]
[0.5, 2.0, -1, -1]


**481. Magical String**

A magical string s consists of only '1' and '2' and obeys the following rules:

The string s is magical because concatenating the number of contiguous occurrences of characters '1' and '2' generates the string s itself.
The first few elements of s is s = "1221121221221121122……". If we group the consecutive 1's and 2's in s, it will be "1 22 11 2 1 22 1 22 11 2 11 22 ......" and the occurrences of 1's or 2's in each group are "1 2 2 1 1 2 1 2 2 1 2 2 ......". You can see that the occurrence sequence is s itself.

Given an integer n, return the number of 1's in the first n number in the magical string s.

https://leetcode.com/problems/magical-string/

In [4]:
def magicalString(n: int) -> int:
    arr = [1, 2, 2]
    i = 2
    while len(arr) < n:
        arr.extend([2 - (i+1) % 2] * arr[i])
        i += 1
    return arr[:n].count(1)

print(magicalString(6)) # 3
print(magicalString(1)) # 1

3
1


**682. Baseball Game**

You are keeping the scores for a baseball game with strange rules. At the beginning of the game, you start with an empty record.

You are given a list of strings operations, where operations[i] is the ith operation you must apply to the record and is one of the following:

* An integer x.
Record a new score of x.
* '+'.
Record a new score that is the sum of the previous two scores.
* 'D'.
Record a new score that is the double of the previous score.
* 'C'.
Invalidate the previous score, removing it from the record.

Return the sum of all the scores on the record after applying all the operations.

https://leetcode.com/problems/baseball-game/

In [3]:
def calPoints(operations: list[str]) -> int:
    ans = []
    for o in operations:
        if o == "+":
            ans.append(ans[-1] + ans[-2])
        elif o == "D":
            ans.append(ans[-1] * 2)
        elif o == "C":
            ans.pop()
        else:
            ans.append(int(o))
    return sum(ans)

print(calPoints(["5","-2","4","C","D","9","+","+"])) # 27
print(calPoints(["1","C"])) # 0
print(calPoints(["5","2","C","D","+"])) # 30

27
0
30


**692. Top K Frequent Words**

Given an array of strings words and an integer k, return the k most frequent strings.

Return the answer sorted by the frequency from highest to lowest. Sort the words with the same frequency by their lexicographical order.

https://leetcode.com/problems/top-k-frequent-words/

In [25]:
# one-liner, but slow
from collections import Counter
def topKFrequent(words: list[str], k: int) -> list[str]:
    return [item[0] for item in Counter(sorted(words)).most_common(k)]

# equally slow
def topKFrequent(words: list[str], k: int) -> list[str]:
    counted = {}
    for w in words:
        counted[w] = counted.get(w, 0) + 1
        
    counted = [(k,v) for k,v in counted.items()]
    counted.sort(key = lambda x: (-x[1], x[0]))
    return [item[0] for item in counted[:k]]

# using heap - seemingly as slow as the above
import heapq
def topKFrequent(words: list[str], k: int) -> list[str]:
    counted = Counter(words)
    heap = [(-count, word) for word, count in counted.items()]
    heapq.heapify(heap)
    return [heapq.heappop(heap)[1] for _ in range(k)]

print(topKFrequent(words = ["i","love","leetcode","i","love","coding"], k = 2)) # ["i","love"]
print(topKFrequent(words = ["the","day","is","sunny","the","the","the","sunny","is","is"], k = 4)) # ["the","is","sunny","day"]

['i', 'love']
['the', 'is', 'sunny', 'day']


**785. Is Graph Bipartite?**

There is an undirected graph with n nodes, where each node is numbered between 0 and n - 1. You are given a 2D array graph, where graph[u] is an array of nodes that node u is adjacent to. More formally, for each v in graph[u], there is an undirected edge between node u and node v. The graph has the following properties:

* There are no self-edges (graph[u] does not contain u).
* There are no parallel edges (graph[u] does not contain duplicate values).
* If v is in graph[u], then u is in graph[v] (the graph is undirected).
* The graph may not be connected, meaning there may be two nodes u and v such that there is no path between them.

A graph is bipartite if the nodes can be partitioned into two independent sets A and B such that every edge in the graph connects a node in set A and a node in set B.

Return true if and only if it is bipartite.

https://leetcode.com/problems/is-graph-bipartite/

In [20]:
def isBipartite(graph: list[list[int]]) -> bool:
    colours = [-1] * len(graph)

    def dfs(n):
        colours[n] = 0
        stack = [n]
        while stack:
            node = stack.pop()
            for edge in graph[node]:
                if colours[edge] == colours[node]:
                    return False
                if colours[edge] == -1:
                    colours[edge] = (colours[node] + 1) % 2
                    stack.append(edge)
        return True

    for i in range(len(graph)):
        if colours[i] == -1:
            if not dfs(i):
                return False

    return True


print(isBipartite(graph = [[1,2,3],[0,2],[0,1,3],[0,2]])) # False
print(isBipartite(graph = [[1,3],[0,2],[1,3],[0,2]])) # True
print(isBipartite([[],[2,4,6],[1,4,8,9],[7,8],[1,2,8,9],[6,9],[1,5,7,8,9],[3,6,9],[2,3,4,6,9],[2,4,5,6,7,8]])) # False

False
True
False


**813. Largest Sum of Averages**

You are given an integer array nums and an integer k. You can partition the array into at most k non-empty adjacent subarrays. The score of a partition is the sum of the averages of each subarray.

Note that the partition must use every integer in nums, and that the score is not necessarily an integer.

Return the maximum score you can achieve of all the possible partitions. Answers within 10^-6 of the actual answer will be accepted.

https://leetcode.com/problems/largest-sum-of-averages/

In [97]:
# times out
def largestSumOfAverages(nums: list[int], k: int) -> float:
    cache = {}
    largest_sum = 0
    l = len(nums)

    def calculate(start_ind, end_ind):
        if (start_ind, end_ind) not in cache:
            cache[(start_ind, end_ind)] = sum(nums[start_ind:end_ind]) / (end_ind - start_ind)
        return cache[(start_ind, end_ind)]

    def helper(total, index, remaining):
        if remaining == 1:
            nonlocal largest_sum
            largest_sum = max(largest_sum, total+calculate(index, l))
        else:
            for i in range(index+1, l-remaining+2):
                helper(total+calculate(index, i), i, remaining-1)

    helper(0, 0, k)

    return largest_sum


def largestSumOfAverages(nums: list[int], k: int) -> float:
    l = len(nums)

    dp = [[float("-inf")] * (l+1)] + [[0] * (l+1) for _ in range(k)]
    dp[0][-1] = 0

    def mean(arr):
        return sum(arr) / len(arr)

    for i in range(1, k+1):
        for j in range(l-i, k-1-i, -1):
            maximum = 0
            for r in range(j+1, l+2-i):
                maximum = max(maximum, mean(nums[j:r]) + dp[i-1][r])
            dp[i][j] = maximum

    return dp[-1][0]




print(largestSumOfAverages(nums = [9,1,2,3,9], k = 3)) # 20.00000
print(largestSumOfAverages(nums = [1,2,3,4,5,6,7], k = 4)) # 20.50000
print(largestSumOfAverages(nums = [4,1,7,5,6,2,3], k = 4)) # 18.16667
long_nums = [4663,3020,7789,1627,9668,1356,4207,1133,8765,4649,205,6455,8864,3554,3916,5925,3995,4540,3487,5444,8259,8802,6777,7306,989,4958,2921,8155,4922,2469,6923,776,9777,1796,708,786,3158,7369,8715,2136,2510,3739,6411,7996,6211,8282,4805,236,1489,7698]
# print(largestSumOfAverages(nums = long_nums, k = 27)) # shouldn't time out

20.0
20.5
18.166666666666664


**841. Keys and Rooms**

There are n rooms labeled from 0 to n - 1 and all the rooms are locked except for room 0. Your goal is to visit all the rooms. However, you cannot enter a locked room without having its key.

When you visit a room, you may find a set of distinct keys in it. Each key has a number on it, denoting which room it unlocks, and you can take all of them with you to unlock the other rooms.

Given an array rooms where rooms[i] is the set of keys that you can obtain if you visited room i, return true if you can visit all the rooms, or false otherwise.

https://leetcode.com/problems/keys-and-rooms/

In [9]:
def canVisitAllRooms(rooms: list[list[int]]) -> bool:
    stack = [0]
    visited = {0}
    while stack:
        r = stack.pop()
        for k in rooms[r]:
            if k not in visited:
                stack.append(k)
                visited.add(k)
    return len(visited) == len(rooms)

print(canVisitAllRooms(rooms = [[1],[2],[3],[]])) # True
print(canVisitAllRooms(rooms = [[1,3],[3,0,1],[2],[0]])) # False

True
False


**1035. Uncrossed Lines**

You are given two integer arrays nums1 and nums2. We write the integers of nums1 and nums2 (in the order they are given) on two separate horizontal lines.

We may draw connecting lines: a straight line connecting two numbers nums1[i] and nums2[j] such that:

* nums1[i] == nums2[j], and
* the line we draw does not intersect any other connecting (non-horizontal) line.

Note that a connecting line cannot intersect even at the endpoints (i.e., each number can only belong to one connecting line).

Return the maximum number of connecting lines we can draw in this way.

https://leetcode.com/problems/uncrossed-lines/

In [8]:
# longest common subsequence
def maxUncrossedLines(nums1: list[int], nums2: list[int]) -> int:
    dp = [[0] * (len(nums1) + 1) for _ in range(len(nums2) + 1)]
    
    for r in range(len(nums2)-1, -1, -1):
        for c in range(len(nums1)-1, -1, -1):
            if nums1[c] == nums2[r]:
                dp[r][c] = dp[r+1][c+1] + 1
            else:
                dp[r][c] = max(dp[r+1][c], dp[r][c+1])
    
    return dp[0][0]


print(maxUncrossedLines(nums1 = [1,4,2], nums2 = [1,2,4])) # 2
print(maxUncrossedLines(nums1 = [2,5,1,2,5], nums2 = [10,5,2,1,5,2])) # 3
print(maxUncrossedLines(nums1 = [1,3,7,1,7,5], nums2 = [1,9,2,5,1])) # 2

2
3
2


**1137. N-th Tribonacci Number**

The Tribonacci sequence Tn is defined as follows: 

T0 = 0, T1 = 1, T2 = 1, and Tn+3 = Tn + Tn+1 + Tn+2 for n >= 0.

Given n, return the value of Tn.

https://leetcode.com/problems/n-th-tribonacci-number/

In [2]:
def tribonacci(n: int) -> int:
    trib = [0,1,1]
    if n < 3:
        return trib[n]
    for _ in range(n-2):
        trib = [trib[1], trib[2], sum(trib)]
    return trib[-1]

print(tribonacci(4)) # 4
print(tribonacci(25)) # 1389537

4
1389537


**1222. Queens That Can Attack the King**

On a 0-indexed 8 x 8 chessboard, there can be multiple black queens ad one white king.

You are given a 2D integer array queens where queens[i] = [xQueeni, yQueeni] represents the position of the ith black queen on the chessboard. You are also given an integer array king of length 2 where king = [xKing, yKing] represents the position of the white king.

Return the coordinates of the black queens that can directly attack the king. You may return the answer in any order.

https://leetcode.com/problems/queens-that-can-attack-the-king/

In [5]:
def queensAttacktheKing(queens: list[list[int]], king: list[int]) -> list[list[int]]:
    queens = set(map(tuple, queens))
    directions = [(0,1), (1,1), (1,0), (0,-1), (-1,-1), (-1,0), (1,-1), (-1,1)]
    ans = []
    for x,y in directions:
        r,c = king
        while 0 <= r < 8 and 0 <= c < 8:
            r += x
            c += y
            if (r,c) in queens:
                ans.append([r,c])
                break
    return ans

print(queensAttacktheKing(queens = [[0,1],[1,0],[4,0],[0,4],[3,3],[2,4]], king = [0,0])) # [[0,1],[1,0],[3,3]]
print(queensAttacktheKing(queens = [[0,0],[1,1],[2,2],[3,4],[3,5],[4,4],[4,5]], king = [3,3])) # [[2,2],[3,4],[4,4]]

[[0, 1], [3, 3], [1, 0]]
[[3, 4], [4, 4], [2, 2]]


**1456. Maximum Number of Vowels in a Substring of Given Length**

Given a string s and an integer k, return the maximum number of vowel letters in any substring of s with length k.

Vowel letters in English are 'a', 'e', 'i', 'o', and 'u'.

https://leetcode.com/problems/maximum-number-of-vowels-in-a-substring-of-given-length/

In [8]:
from collections import deque
def maxVowels(s: str, k: int) -> int:
    maximum = 0
    current = 0
    vowels = {"a", "e", "i", "o", "u"}
    q = deque([])
    for c in s:
        if len(q) == k:
            l = q.popleft()
            if l in vowels:
                current -= 1
        q.append(c)
        if c in vowels:
            current += 1
            maximum = max(maximum, current)
    return maximum

print(maxVowels(s = "abciiidef", k = 3)) # 3
print(maxVowels(s = "aeiou", k = 2)) # 2
print(maxVowels(s = "leetcode", k = 3)) # 2

3
2
2


**1498. Number of Subsequences That Satisfy the Given Sum Condition**

You are given an array of integers nums and an integer target.

Return the number of non-empty subsequences of nums such that the sum of the minimum and maximum element on it is less or equal to target. Since the answer may be too large, return it modulo 10^9 + 7.

https://leetcode.com/problems/number-of-subsequences-that-satisfy-the-given-sum-condition/

In [34]:
# quite slow
def numSubseq(nums: list[int], target: int) -> int:
    nums.sort()
    
    sum_target = target - nums[0]
    l = 0
    r = len(nums) - 1
    while l <= r:
        m = (l+r) // 2
        if nums[m] <= sum_target:
            l = m + 1
        else:
            r = m - 1
    
    out = 0

    l = 0
    while l <= r:
        out += 2**(r-l)
        l += 1
        while l < len(nums) and nums[l] + nums[r] > target:
            r -= 1

    return out % (10**9 + 7)

# as slow
def numSubseq(nums: list[int], target: int) -> int:
    nums.sort()

    def binary(l, r, t):
        while l <= r:
            m = (l+r) // 2
            if nums[m] <= t:
                l = m + 1
            else:
                r = m - 1
        return r

    out = 0
    l = 0
    r = binary(0, len(nums)-1, target-nums[0])

    while l <= r:
        out += 2**(r-l)
        l += 1
        if l < len(nums):
            r = binary(l, r, target - nums[l])

    return out % (10**9 + 7)

print(numSubseq(nums = [3,5,6,7], target = 9)) # 4
print(numSubseq(nums = [3,3,6,8], target = 10)) # 6
print(numSubseq(nums = [2,3,3,4,6,7], target = 12)) # 61
print(numSubseq(nums = [5,2,4,1,7,6,8], target = 16)) # 127

4
6
61
127


**1518. Water Bottles**

There are numBottles water bottles that are initially full of water. You can exchange numExchange empty water bottles from the market with one full water bottle.

The operation of drinking a full water bottle turns it into an empty bottle.

Given the two integers numBottles and numExchange, return the maximum number of water bottles you can drink.

https://leetcode.com/problems/water-bottles/

In [12]:
def numWaterBottles(numBottles: int, numExchange: int) -> int:
    ans = numBottles
    rem = 0
    while numBottles > 0:
        numBottles += rem
        rem = numBottles % numExchange
        numBottles //= numExchange
        ans += numBottles
    return ans

print(numWaterBottles(numBottles = 9, numExchange = 3)) # 13
print(numWaterBottles(numBottles = 15, numExchange = 4)) # 19

13
19


**1557. Minimum Number of Vertices to Reach All Nodes**

Given a directed acyclic graph, with n vertices numbered from 0 to n-1, and an array edges where edges[i] = [fromi, toi] represents a directed edge from node fromi to node toi.

Find the smallest set of vertices from which all nodes in the graph are reachable. It's guaranteed that a unique solution exists.

Notice that you can return the vertices in any order.

https://leetcode.com/problems/minimum-number-of-vertices-to-reach-all-nodes/

In [3]:
def findSmallestSetOfVertices(n: int, edges: list[list[int]]) -> list[int]:
    return list(set(range(n)) - {e[1] for e in edges})

print(findSmallestSetOfVertices(n = 6, edges = [[0,1],[0,2],[2,5],[3,4],[4,2]])) # [0,3]
print(findSmallestSetOfVertices(n = 5, edges = [[0,1],[2,1],[3,1],[1,4],[2,4]])) # [0,2,3]

[0, 3]
[0, 2, 3]


**1572. Matrix Diagonal Sum**

Given a square matrix mat, return the sum of the matrix diagonals.

Only include the sum of all the elements on the primary diagonal and all the elements on the secondary diagonal that are not part of the primary diagonal.

https://leetcode.com/problems/matrix-diagonal-sum/

In [2]:
def diagonalSum(mat: list[list[int]]) -> int:
    l = 0
    r = len(mat[0]) - 1
    ans = 0
    for i in range(len(mat)):
        ll = l+i
        rr = r-i
        ans += mat[i][ll]
        if ll != rr:
            ans += mat[i][rr]
    return ans

print(diagonalSum(mat = [[1,2,3],
              [4,5,6],
              [7,8,9]])) # 25
print(diagonalSum(mat = [[1,1,1,1],
              [1,1,1,1],
              [1,1,1,1],
              [1,1,1,1]])) # 8
print(diagonalSum(mat = [[5]])) # 5

25
8
5


**1721. Swapping Nodes in a Linked List**

You are given the head of a linked list, and an integer k.

Return the head of the linked list after swapping the values of the kth node from the beginning and the kth node from the end (the list is 1-indexed).

https://leetcode.com/problems/swapping-nodes-in-a-linked-list/

In [26]:
def swapNodes(head: ListNode, k: int) -> ListNode:
    if not head.next:
        return head
    slow = head
    fast = head.next
    counter = 1
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        counter += 1
    if fast:
        length = counter * 2
    else:
        length = counter * 2 - 1
        if k == counter:
            return head
        
    if k < counter:
        left = head
        left_counter = 1
        right = slow
        right_counter = counter

    else:
        left = slow
        left_counter = counter
        right = head
        right_counter = 1

    while left_counter < k:
        left = left.next
        left_counter += 1
        
    while right_counter < (length - k + 1):
        right = right.next
        right_counter += 1

    left.val, right.val = right.val, left.val

    return head


print_linked_list(swapNodes(make_linked_list([1,2,3,4,5]), 2)) # [1,4,3,2,5]
print_linked_list(swapNodes(make_linked_list([1,2,3,4,5]), 3)) # [1,2,3,4,5]
print_linked_list(swapNodes(make_linked_list([7,9,6,6,7,8,3,0,9,5]), 5)) # [7,9,6,6,8,7,3,0,9,5]
print_linked_list(swapNodes(make_linked_list([100,90]), 2)) # [90,100]

1 4 3 2 5 
1 2 3 4 5 
7 9 6 6 8 7 3 0 9 5 
90 100 


**1799. Maximize Score After N Operations**

You are given nums, an array of positive integers of size 2 * n. You must perform n operations on this array.

In the ith operation (1-indexed), you will:

* Choose two elements, x and y.
* Receive a score of i * gcd(x, y).
* Remove x and y from nums.

Return the maximum score you can receive after performing n operations.

The function gcd(x, y) is the greatest common divisor of x and y.

 https://leetcode.com/problems/maximize-score-after-n-operations/

In [51]:
# via https://leetcode.com/problems/maximize-score-after-n-operations/discuss/3521561/PythonJavaC%2B%2BSimple-SolutionEasy-to-Understand
def maxScore(nums: list[int]) -> int:

    def gcd(x, y):
        while y:
            x, y = y, x%y
        return x
    
    l = len(nums)
    gcds = [[0] * l for _ in range(l)]
    for i in range(l-1):
        for j in range(i+1, l):
            gcds[i][j] = gcd(nums[i], nums[j])


    # here we'll keep the maximum score achieved for each combination
    # 1 << l = 2 ** l
    dp = [0] * (1 << l)

    # iterating through every bitmap between 1 and 2**l
    for state in range(1, 1 << l):

        # if there is an odd number of 1s in the bitmap, we skip this step
        # because we must have an even number of candidates
        cnt = bin(state).count('1')

        if cnt % 2:
            continue

        for i in range(l):
            # this would only evaluate to 0 if 1 isn't yet at the position i
            # this means this candidate has already been used
            if not (state & (1 << i)):
                continue
            for j in range(i+1, l):
                # this would only evaluate to 0 if 1 isn't yet at the position j
                # this means this candidate has already been used
                if not (state & (1 << j)):
                    continue

                # ^ xor operator
                # e.g.: 01110 ^ 01000 ^ 00010 = 00100
                # in decimal: 14 ^ 8 ^ 2 = 4
                # we replace 1s in the current state with 0s at positions i and j
                prevState = state ^ (1 << i) ^ (1 << j)
                
                # print(bin(state), bin(prevState))

                # here we update the dp with 
                dp[state] = max(dp[state], 
                                # cnt // 2 = the number of 1s remaining. 
                                # dividing this number by 2 gives us the number of the operation
                                dp[prevState] + cnt // 2 * gcds[i][j])
                
                # print(*dp, sep="  ") 

    # (1 << l) - 1 will evaluate to 1 * len(nums)
    # e.g. (1 << 6) - 1 = 111111 (63 in decimal)
    return dp[(1 << l) - 1]

print(maxScore([1,2])) # 1
print(maxScore([3,4,6,8])) # 11
print(maxScore([1,2,3,4,5,6])) # 14

1
11
14


**1964. Find the Longest Valid Obstacle Course at Each Position**

You want to build some obstacle courses. You are given a 0-indexed integer array obstacles of length n, where obstacles[i] describes the height of the ith obstacle.

For every index i between 0 and n - 1 (inclusive), find the length of the longest obstacle course in obstacles such that:

* You choose any number of obstacles between 0 and i inclusive.
* You must include the ith obstacle in the course.
* You must put the chosen obstacles in the same order as they appear in obstacles.
* Every obstacle (except the first) is taller than or the same height as the obstacle immediately before it.

Return an array ans of length n, where ans[i] is the length of the longest obstacle course for index i as described above.

https://leetcode.com/problems/find-the-longest-valid-obstacle-course-at-each-position/

In [6]:
# a little slow
def longestObstacleCourseAtEachPosition(obstacles: list[int]) -> list[int]:
    ans = []
    stacks = []
    for n in obstacles:
        l = 0
        r = len(stacks)-1
        while l <= r:
            m = (l+r) // 2
            if stacks[m][-1] <= n:
                l = m + 1
            else:
                r = m - 1
        if l == len(stacks):
            stacks.append([n])
        else:
            stacks[l].append(n)
        ans.append(l+1)
    return ans

# optimised
def longestObstacleCourseAtEachPosition(obstacles: list[int]) -> list[int]:
    ans = []
    dp = []
    for n in obstacles:
        if not dp or n >= dp[-1]:
            dp.append(n)
            ans.append(len(dp))
        else:
            l = 0
            r = len(dp)-1
            while l <= r:
                m = (l+r) // 2
                if dp[m] <= n:
                    l = m + 1
                else:
                    r = m - 1
            dp[l] = n
            ans.append(l+1)
    return ans

print(longestObstacleCourseAtEachPosition([1,2,3,2])) # [1,2,3,3]
print(longestObstacleCourseAtEachPosition([2,2,1])) # [1,2,1]
print(longestObstacleCourseAtEachPosition([3,1,5,6,4,2])) # [1,1,2,3,2,2]

[1, 2, 3, 3]
[1, 2, 1]
[1, 1, 2, 3, 2, 2]


**2006. Count Number of Pairs With Absolute Difference K**

Given an integer array nums and an integer k, return the number of pairs (i, j) where i < j such that |nums[i] - nums[j]| == k.

https://leetcode.com/problems/count-number-of-pairs-with-absolute-difference-k/

In [2]:
from collections import Counter
def countKDifference(nums: list[int], k: int) -> int:
    counter = Counter(nums)
    ans = 0
    for n in counter.keys():
        if n-k in counter:
            ans += counter[n] * counter[n-k]
    return ans

print(countKDifference(nums = [1,2,2,1], k = 1)) # 4
print(countKDifference(nums = [1,3], k = 3)) # 0

4
0


**2079. Watering Plants**

You want to water n plants in your garden with a watering can. The plants are arranged in a row and are labeled from 0 to n - 1 from left to right where the ith plant is located at x = i. There is a river at x = -1 that you can refill your watering can at.

Each plant needs a specific amount of water. You will water the plants in the following way:

* Water the plants in order from left to right.
* After watering the current plant, if you do not have enough water to completely water the next plant, return to the river to fully refill the watering can.
* You cannot refill the watering can early.

You are initially at the river (i.e., x = -1). It takes one step to move one unit on the x-axis.

Given a 0-indexed integer array plants of n integers, where plants[i] is the amount of water the ith plant needs, and an integer capacity representing the watering can capacity, return the number of steps needed to water all the plants.

https://leetcode.com/problems/watering-plants/

In [3]:
def wateringPlants(plants: list[int], capacity: int) -> int:
    counter = len(plants)
    current = capacity
    for i in range(len(plants)):
        current -= plants[i]
        if current < 0:
            current = capacity - plants[i]
            counter += 2 * i
    return counter

print(wateringPlants(plants = [2,2,3,3], capacity = 5)) # 14
print(wateringPlants(plants = [1,1,1,4,2,3], capacity = 4)) # 30
print(wateringPlants(plants = [7,7,7,7,7,7,7], capacity = 8)) # 49

14
30
49


**2114. Maximum Number of Words Found in Sentences**

A sentence is a list of words that are separated by a single space with no leading or trailing spaces.

You are given an array of strings sentences, where each sentences[i] represents a single sentence.

Return the maximum number of words that appear in a single sentence.

https://leetcode.com/problems/maximum-number-of-words-found-in-sentences/

In [1]:
def mostWordsFound(sentences: list[str]) -> int:
    return max([len(s.split()) for s in sentences])

print(mostWordsFound(["alice and bob love leetcode", "i think so too", "this is great thanks very much"])) # 6
print(mostWordsFound(["please wait", "continue to fight", "continue to win"])) # 3

6
3


**2130. Maximum Twin Sum of a Linked List**

In a linked list of size n, where n is even, the ith node (0-indexed) of the linked list is known as the twin of the (n-1-i)th node, if 0 <= i <= (n / 2) - 1.

For example, if n = 4, then node 0 is the twin of node 3, and node 1 is the twin of node 2. These are the only nodes with twins for n = 4.

The twin sum is defined as the sum of a node and its twin.

Given the head of a linked list with even length, return the maximum twin sum of the linked list.

https://leetcode.com/problems/maximum-twin-sum-of-a-linked-list/

In [5]:
def pairSum(head: ListNode) -> int:
    sums = [head.val]
    slow = head
    fast = head.next
    while fast.next:
        slow = slow.next
        fast = fast.next.next
        sums.append(slow.val)
    
    maximum = 0
    i = len(sums)-1
    
    slow = slow.next
    while slow:
        sums[i] += slow.val
        maximum = max(maximum, sums[i])
        i -= 1
        slow = slow.next
        
    return maximum

**2140. Solving Questions With Brainpower**

You are given a 0-indexed 2D integer array questions where questions[i] = [pointsi, brainpoweri].

The array describes the questions of an exam, where you have to process the questions in order (i.e., starting from question 0) and make a decision whether to solve or skip each question. Solving question i will earn you pointsi points but you will be unable to solve each of the next brainpoweri questions. If you skip question i, you get to make the decision on the next question.

For example, given questions = [[3, 2], [4, 3], [4, 4], [2, 5]]:
If question 0 is solved, you will earn 3 points but you will be unable to solve questions 1 and 2.
If instead, question 0 is skipped and question 1 is solved, you will earn 4 points but you will be unable to solve questions 2 and 3.
Return the maximum points you can earn for the exam.

https://leetcode.com/problems/solving-questions-with-brainpower/

In [11]:
def mostPoints(questions: list[list[int]]) -> int:
    l = len(questions)
    dp = [0] * (l + 1)
    for i in range(l-1,-1,-1):
        if i + questions[i][1] < l:
            dp[i] = max(questions[i][0] + dp[i + questions[i][1] + 1], dp[i+1])
        else:
            dp[i] = max(questions[i][0], dp[i+1])
    return dp[0]

print(mostPoints([[3,2],[4,3],[4,4],[2,5]])) # 5
print(mostPoints([[1,1],[2,2],[3,3],[4,4],[5,5]])) # 7

5
7


**2466. Count Ways To Build Good Strings**

Given the integers zero, one, low, and high, we can construct a string by starting with an empty string, and then at each step perform either of the following:

* Append the character '0' zero times.
* Append the character '1' one times.

This can be performed any number of times.

A good string is a string constructed by the above process having a length between low and high (inclusive).

Return the number of different good strings that can be constructed satisfying these properties. Since the answer can be large, return it modulo 10^9 + 7.

https://leetcode.com/problems/count-ways-to-build-good-strings/

In [7]:
def countGoodStrings(low: int, high: int, zero: int, one: int) -> int:
    dp = [1] + [0] * (high)
    for i in range(1, len(dp)):
        if i >= zero:
            dp[i] += (dp[i-zero])           
        if i >= one:
            dp[i] += (dp[i-one])
    print(dp)
    return sum(dp[low:]) % (10**9 + 7)

print(countGoodStrings(low = 3, high = 3, zero = 1, one = 1)) # 8
print(countGoodStrings(low = 2, high = 3, zero = 1, one = 2)) # 5

[1, 2, 4, 8]
8
[1, 1, 2, 3]
5


**2540. Minimum Common Value**

Given two integer arrays nums1 and nums2, sorted in non-decreasing order, return the minimum integer common to both arrays. If there is no common integer amongst nums1 and nums2, return -1.

Note that an integer is said to be common to nums1 and nums2 if both arrays have at least one occurrence of that integer.

https://leetcode.com/problems/minimum-common-value/

In [15]:
# one liner
def getCommon(nums1: list[int], nums2: list[int]) -> int:
    return min((set(nums1) & set(nums2)) or [-1])

def getCommon(nums1: list[int], nums2: list[int]) -> int:
    nums1 = set(nums1)
    for n in nums2:
        if n in nums1:
            return n
    return -1

print(getCommon(nums1 = [1,2,3], nums2 = [2,4])) # 2
print(getCommon(nums1 = [1,2,3,6], nums2 = [2,3,4,5])) # 2

2
2


**2582. Pass the Pillow**

There are n people standing in a line labeled from 1 to n. The first person in the line is holding a pillow initially. Every second, the person holding the pillow passes it to the next person standing in the line. Once the pillow reaches the end of the line, the direction changes, and people continue passing the pillow in the opposite direction.

For example, once the pillow reaches the nth person they pass it to the n - 1th person, then to the n - 2th person and so on.

Given the two positive integers n and time, return the index of the person holding the pillow after time seconds.

https://leetcode.com/problems/pass-the-pillow/

In [28]:
def passThePillow(n: int, time: int) -> int:
    k = time % ((n-1) * 2) + 1
    return k if k <= n else n - (k-n)

print(passThePillow(4,5)) # 2
print(passThePillow(3,2)) # 3

2
3


**2598. Smallest Missing Non-negative Integer After Operations**

You are given a 0-indexed integer array nums and an integer value.

In one operation, you can add or subtract value from any element of nums.

* For example, if nums = [1,2,3] and value = 2, you can choose to subtract value from nums[0] to make nums = [-1,2,3].

The MEX (minimum excluded) of an array is the smallest missing non-negative integer in it.

* For example, the MEX of [-1,2,3] is 0 while the MEX of [1,0,3] is 2.

Return the maximum MEX of nums after applying the mentioned operation any number of times.

https://leetcode.com/problems/smallest-missing-non-negative-integer-after-operations/

In [46]:
# slowish
def findSmallestInteger(nums: list[int], value: int) -> int:
    bools = [[False, i] for i in range (len(nums) + 1)]
    for n in nums:
        m = n % value
        if m <= len(nums):
            if not bools[m][0]:
                bools[m][0] = True
            else:
                bools[m][1] += value
                if bools[m][1] <= len(nums):
                    bools[bools[m][1]][0] = True
    for i in range(len(bools)):
        if not bools[i][0]:
            return i
        
# another solution
from collections import Counter
def findSmallestInteger(nums: list[int], value: int) -> int:
    counts = Counter([n%value for n in nums])
    for i in range(len(nums) + 1):
        if counts[m := i % value] == 0:
            return i
        else:
            counts[m] -= 1
        
print(findSmallestInteger(nums = [1,-10,7,13,6,8], value = 5)) # 4
print(findSmallestInteger(nums = [1,-10,7,13,6,8], value = 7)) # 2
print(findSmallestInteger(nums = [3,0,3,2,4,2,1,1,0,4], value = 5)) # 10

4
2
10


**2640. Find the Score of All Prefixes of an Array**

We define the conversion array conver of an array arr as follows:

conver[i] = arr[i] + max(arr[0..i]) where max(arr[0..i]) is the maximum value of arr[j] over 0 <= j <= i.
We also define the score of an array arr as the sum of the values of the conversion array of arr.

Given a 0-indexed integer array nums of length n, return an array ans of length n where ans[i] is the score of the prefix nums[0..i].

 https://leetcode.com/problems/find-the-score-of-all-prefixes-of-an-array/

In [11]:
def findPrefixScore(nums: list[int]) -> list[int]:
    maximum = 0
    total = 0
    ans = []
    for n in nums:
        maximum = max(n, maximum)
        n += maximum
        total += n
        ans.append(total)
    return ans

print(findPrefixScore(nums = [2,3,7,5,10])) # [4,10,24,36,56]
print(findPrefixScore(nums = [1,1,2,4,8,16])) # [2,4,8,16,32,64]

[4, 10, 24, 36, 56]
[2, 4, 8, 16, 32, 64]


**2643. Row With Maximum Ones**

Given a m x n binary matrix mat, find the 0-indexed position of the row that contains the maximum count of ones, and the number of ones in that row.

In case there are multiple rows that have the maximum count of ones, the row with the smallest row number should be selected.

Return an array containing the index of the row, and the number of ones in it.

https://leetcode.com/problems/row-with-maximum-ones/

In [17]:
def rowAndMaximumOnes(mat: list[list[int]]) -> list[int]:
    l = len(mat)
    row = 0
    maximum = 0
    for i in range(l):
        if (s := sum(mat[i])) > maximum:
            maximum = s
            row = i
    return [row, maximum]

print(rowAndMaximumOnes([[0,1],[1,0]])) # [0,1]
print(rowAndMaximumOnes([[0,0,0],[0,1,1]])) # [1,2]
print(rowAndMaximumOnes([[0,0],[1,1],[0,0]])) # [1,2]

[0, 1]
[1, 2]
[1, 2]


**2644. Find the Maximum Divisibility Score**

You are given two 0-indexed integer arrays nums and divisors.

The divisibility score of divisors[i] is the number of indices j such that nums[j] is divisible by divisors[i].

Return the integer divisors[i] with the maximum divisibility score. If there is more than one integer with the maximum score, return the minimum of them.

https://leetcode.com/problems/find-the-maximum-divisibility-score/

In [16]:
from collections import Counter
def maxDivScore(nums: list[int], divs: list[int]) -> int:
    nums = Counter(nums)
    divs = {d:0 for d in divs}
    maxscore = 0
    mindiv = min(divs.keys())
    for n in nums.keys():
        for d in divs.keys():
            if not n % d:
                divs[d] += nums[n]
                if maxscore == divs[d]:
                    mindiv = min(mindiv, d)
                elif maxscore < divs[d]:
                    maxscore = divs[d]
                    mindiv = d
    return mindiv

print(maxDivScore(nums = [4,7,9,3,9], divs = [5,2,3])) # 3
print(maxDivScore(nums = [20,14,21,10], divs = [5,7,5])) # 5
print(maxDivScore(nums = [12], divs = [10,16])) # 10

3
5
10


**2651. Calculate Delayed Arrival Time**

You are given a positive integer arrivalTime denoting the arrival time of a train in hours, and another positive integer delayedTime denoting the amount of delay in hours.

Return the time when the train will arrive at the station.

Note that the time in this problem is in 24-hours format.

https://leetcode.com/problems/calculate-delayed-arrival-time/

In [13]:
def findDelayedArrivalTime(a: int, d: int) -> int:
    return (a+d) % 24

print(findDelayedArrivalTime(a = 15, d = 5 )) # 20
print(findDelayedArrivalTime(a = 13, d = 11)) # 0

20
0


**2652. Sum Multiples**

Given a positive integer n, find the sum of all integers in the range [1, n] inclusive that are divisible by 3, 5, or 7.

Return an integer denoting the sum of all numbers in the given range satisfying the constraint.

https://leetcode.com/problems/sum-multiples/

In [26]:
def sumOfMultiples(n: int) -> int:
    ans = 0
    s = set()
    for m in [3,5,7]:
        for k in range(m, n+1, m):
            if k not in s:
                s.add(k)
                ans += k
    return ans
        
print(sumOfMultiples(7)) # 21
print(sumOfMultiples(10)) # 40
print(sumOfMultiples(9)) # 30

21
40
30


**2656. Maximum Sum With Exactly K Elements**

You are given a 0-indexed integer array nums and an integer k. Your task is to perform the following operation exactly k times in order to maximize your score:

1. Select an element m from nums.
2. Remove the selected element m from the array.
3. Add a new element with a value of m + 1 to the array.
4. Increase your score by m.

Return the maximum score you can achieve after performing the operation exactly k times.

https://leetcode.com/problems/maximum-sum-with-exactly-k-elements/

In [11]:
def maximizeSum(nums: list[int], k: int) -> int:
    m = max(nums)
    return sum([m+n for n in range(k)])

print(maximizeSum(nums = [1,2,3,4,5], k = 3)) # 18
print(maximizeSum(nums = [5,5,5], k = 2)) # 11

18
11


**2660. Determine the Winner of a Bowling Game**

You are given two 0-indexed integer arrays player1 and player2, that represent the number of pins that player 1 and player 2 hit in a bowling game, respectively.

The bowling game consists of n turns, and the number of pins in each turn is exactly 10.

Assume a player hit xi pins in the ith turn. The value of the ith turn for the player is:

* 2*xi if the player hit 10 pins in any of the previous two turns.
* Otherwise, It is xi.

The score of the player is the sum of the values of their n turns.

Return

* 1 if the score of player 1 is more than the score of player 2,
* 2 if the score of player 2 is more than the score of player 1, and
* 0 in case of a draw.

https://leetcode.com/problems/determine-the-winner-of-a-bowling-game/

In [11]:
def isWinner(player1: list[int], player2: list[int]) -> int:
    def calculate(arr):
        score = 0
        doubles = 0
        for n in arr:
            score += 2 * n if doubles > 0 else n
            doubles -= 1
            if n == 10:
                doubles = 2
        return score
    score1 = calculate(player1)
    score2 = calculate(player2)
    if score1 == score2:
        return 0
    return 1 if score1 > score2 else 2

print(isWinner(player1 = [4,10,7,9], player2 = [6,5,2,3])) # 1
print(isWinner(player1 = [3,5,7,6], player2 = [8,10,10,2])) # 2
print(isWinner(player1 = [2,3], player2 = [4,1])) # 0

1
2
0


**2670. Find the Distinct Difference Array**

You are given a 0-indexed array nums of length n.

The distinct difference array of nums is an array diff of length n such that diff[i] is equal to the number of distinct elements in the suffix nums[i + 1, ..., n - 1] subtracted from the number of distinct elements in the prefix nums[0, ..., i].

Return the distinct difference array of nums.

Note that nums[i, ..., j] denotes the subarray of nums starting at index i and ending at index j inclusive. Particularly, if i > j then nums[i, ..., j] denotes an empty subarray.

https://leetcode.com/problems/find-the-distinct-difference-array/

In [25]:
from collections import Counter
def distinctDifferenceArray(nums: list[int]) -> list[int]:
    counts = Counter(nums)
    r = len(counts)
    ans = []
    s = set()
    for i in range(len(nums)):
        counts[nums[i]] -= 1
        if counts[nums[i]] == 0:
            r -= 1
        s.add(nums[i])
        ans.append(len(s)-r)
    return ans

print(distinctDifferenceArray(nums = [1,2,3,4,5])) # [-3,-1,1,3,5]
print(distinctDifferenceArray(nums = [3,2,3,4,2])) # [-2,-1,0,2,3]

[-3, -1, 1, 3, 5]
[-2, -1, 0, 2, 3]


**2678. Number of Senior Citizens**

You are given a 0-indexed array of strings details. Each element of details provides information about a given passenger compressed into a string of length 15. The system is such that:

* The first ten characters consist of the phone number of passengers.
* The next character denotes the gender of the person.
* The following two characters are used to indicate the age of the person.
* The last two characters determine the seat allotted to that person.

Return the number of passengers who are strictly more than 60 years old.

https://leetcode.com/problems/number-of-senior-citizens/

In [22]:
def countSeniors(details: list[str]) -> int:
    return sum([s[-4:-2] > "60" for s in details])

print(countSeniors(details = ["7868190130M7522","5303914400F9211","9273338290F4010"])) # 2

2


**2682. Find the Losers of the Circular Game**

There are n friends that are playing a game. The friends are sitting in a circle and are numbered from 1 to n in clockwise order. More formally, moving clockwise from the ith friend brings you to the (i+1)th friend for 1 <= i < n, and moving clockwise from the nth friend brings you to the 1st friend.

The rules of the game are as follows:

1st friend receives the ball.

* After that, 1st friend passes it to the friend who is k steps away from them in the clockwise direction.
* After that, the friend who receives the ball should pass it to the friend who is 2 * k steps away from them in the clockwise direction.
* After that, the friend who receives the ball should pass it to the friend who is 3 * k steps away from them in the clockwise direction, and so on and so forth.

In other words, on the ith turn, the friend holding the ball should pass it to the friend who is i * k steps away from them in the clockwise direction.

The game is finished when some friend receives the ball for the second time.

The losers of the game are friends who did not receive the ball in the entire game.

Given the number of friends, n, and an integer k, return the array answer, which contains the losers of the game in the ascending order.

https://leetcode.com/problems/find-the-losers-of-the-circular-game/

In [2]:
def circularGameLosers(n: int, k: int) -> list[int]:
    friends = set(range(1, n+1))
    turn = 1
    kk = k
    while True:
        if turn in friends:
            friends.remove(turn)
        else:
            return list(friends)
        turn = (turn + kk) % n or n
        kk += k

print(circularGameLosers(n = 5, k = 2)) # [4,5]
print(circularGameLosers(n = 4, k = 4)) # [2,3,4]

[4, 5]
[2, 3, 4]


**2696. Minimum String Length After Removing Substrings**

You are given a string s consisting only of uppercase English letters.

You can apply some operations to this string where, in one operation, you can remove any occurrence of one of the substrings "AB" or "CD" from s.

Return the minimum possible length of the resulting string that you can obtain.

Note that the string concatenates after removing the substring and could produce new "AB" or "CD" substrings.

https://leetcode.com/problems/minimum-string-length-after-removing-substrings/

In [10]:
def minLength(s: str) -> int:
    stack = []
    for c in s:
        if (stack and 
            ((c == "B" and stack[-1] == "A") or 
                (c == "D" and stack[-1] == "C"))):
            stack.pop()
        else:
            stack.append(c)
    return len(stack)

print(minLength("ABFCACDB")) # 2
print(minLength("ACBBD")) # 5

2
5


**2697. Lexicographically Smallest Palindrome**

You are given a string s consisting of lowercase English letters, and you are allowed to perform operations on it. In one operation, you can replace a character in s with another lowercase English letter.

Your task is to make s a palindrome with the minimum number of operations possible. If there are multiple palindromes that can be made using the minimum number of operations, make the lexicographically smallest one.

A string a is lexicographically smaller than a string b (of the same length) if in the first position where a and b differ, string a has a letter that appears earlier in the alphabet than the corresponding letter in b.

Return the resulting palindrome string.

https://leetcode.com/problems/lexicographically-smallest-palindrome/

In [8]:
def makeSmallestPalindrome(s: str) -> str:
    s = list(s)
    l = 0
    r = len(s) - 1
    while l < r:
        s[l] = s[r] = min(s[l], s[r])
        l += 1
        r -= 1
    return "".join(s)

print(makeSmallestPalindrome("egcfe")) # "efcfe"
print(makeSmallestPalindrome("abcd")) # "abba"
print(makeSmallestPalindrome("seven")) # "neven"

efcfe
abba
neven
